In [1]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape
from scipy.spatial import ConvexHull

# 读取浙江省的边界数据
zhejiang_boundary = gpd.read_file(r'C:\Users\21523\Desktop\全国乡镇边界的经纬度数据集\浙江省行政区域经纬度数据集\scaled_40_percent_浙江省.geoJson')

# 读取数据集，并随机选择10000条数据
data = pd.read_csv(r'D:\第四篇论文数据\第二篇论文编写程序\人类行为时空特性关联性建模\浙江省手机信令数据集\visit_duration_distance_v.csv', usecols=['slon', 'slat']).sample(n=10000)

# 创建地图对象
m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

# 创建一个字典来存储不同地级市的凸包点数据
city_hull_points = {}

# 遍历各地级市的边界数据
for index, row in zhejiang_boundary.iterrows():
    city_name = row['name']
    city_boundary = shape(row['geometry'])
    
    # 如果数据点为空，跳过绘制
    if data.empty:
        continue
    
    # 创建几何对象的缓冲区来避免拓扑错误，并放大凸包
    city_boundary_buffered = city_boundary.buffer(0)
    city_boundary_expanded = city_boundary_buffered.buffer(city_boundary_buffered.bounds[1] * 0.0035)
    
    # 筛选位于当前地级市边界内的数据点
    points_in_city = data[data.apply(lambda x: city_boundary_expanded.contains(Point(x['slon'], x['slat'])), axis=1)]
    
    # 如果数据点为空，跳过绘制
    if points_in_city.empty:
        continue
    
    # 获取数据点的坐标
    coords = points_in_city[['slon', 'slat']].values
    
    # 计算凸包
    hull = ConvexHull(coords)
    
    # 将凸包点转换为多边形
    hull_polygon = folium.Polygon(locations=[(coords[i][1], coords[i][0]) for i in hull.vertices], fill=True, fill_color='#3186cc', fill_opacity=0.5, color='#3186cc', weight=1)  # 增加轮廓线的粗细
    
    # 将凸包多边形添加到地图
    hull_polygon.add_to(m)
    
    # 将凸包点数据存储在字典中
    city_hull_points[city_name] = points_in_city[['slon', 'slat']]

# 将不同地级市的凸包点数据存储在不同的数据帧中
for city_name, points_df in city_hull_points.items():
    points_df.to_csv(f'{city_name}_convex_hull_points.csv', index=False)

# 展示地图
m

In [90]:
import pandas as pd

# 读取 CSV 文件
file_path = r"C:\Users\21523\Desktop\ok_geo.csv\ok_geo.csv"
data = pd.read_csv(file_path)

# 提取第910条到第1009条数据到新的数据集 zhejiang
zhejiang = data.iloc[909:1010]
zhejiang = zhejiang[zhejiang['deep'] == 2]

# 修改索引为区的名称
zhejiang.index = zhejiang['name']

# 选择要保存的列
zhejiang_boundary = zhejiang[['polygon']]

# 保存到新的Excel文件
new_file_path = r"C:\Users\21523\Desktop\zhejiang_boundary.xlsx"
zhejiang_boundary.to_excel(new_file_path)

print("保存成功！")

保存成功！


In [2]:
zhejiang_boundary = gpd.read_file(r'C:\Users\21523\Desktop\converted_zhejiang_boundary_safe.geojson')

In [3]:
zhejiang_boundary

,name,geometry
0,上城区,"POLYGON ((120.13711 30.19809, 120.13716 30.199..."
1,拱墅区,"POLYGON ((120.08692 30.33302, 120.08782 30.333..."
2,西湖区,"POLYGON ((119.99634 30.18154, 120.00131 30.188..."
3,滨江区,"POLYGON ((120.12439 30.17220, 120.12463 30.177..."
4,萧山区,"POLYGON ((120.07713 29.88752, 120.07716 29.887..."
...,...,...
85,松阳县,"POLYGON ((119.17226 28.30330, 119.17368 28.306..."
86,云和县,"POLYGON ((119.35398 27.97945, 119.35482 27.981..."
87,庆元县,"POLYGON ((118.83593 27.82947, 118.83647 27.830..."
88,景宁畲族自治县,"POLYGON ((119.23581 27.80097, 119.23605 27.805..."


In [4]:
import geopandas as gpd

# 读取浙江省的边界数据
zhejiang_boundary = gpd.read_file(r'C:\Users\21523\Desktop\converted_zhejiang_boundary_safe.geojson')

# 遍历各地级市的边界数据
for index, row in zhejiang_boundary.iterrows():
    city_name = row['name']
    city_geometry = row['geometry']
    
    # 检查几何信息是否为空
    if city_geometry is None:
        print(f"城市 {city_name} 的几何信息为空！")
    else:
        # 尝试创建几何对象
        try:
            city_boundary = shape(city_geometry)
            # 在这里可以进行其他操作，比如绘制地图等
        except Exception as e:
            print(f"无法创建城市 {city_name} 的几何对象：{e}")

城市 淳安县 的几何信息为空！
城市 永嘉县 的几何信息为空！
城市 越城区 的几何信息为空！
城市 柯桥区 的几何信息为空！
城市 上虞区 的几何信息为空！
城市 诸暨市 的几何信息为空！
城市 嵊州市 的几何信息为空！
城市 东阳市 的几何信息为空！
城市 衢江区 的几何信息为空！
城市 江山市 的几何信息为空！
城市 临海市 的几何信息为空！
城市 缙云县 的几何信息为空！


In [5]:
import geopandas as gpd

# 读取浙江省的边界数据
zhejiang_boundary = gpd.read_file(r'C:\Users\21523\Desktop\converted_zhejiang_boundary_safe.geojson')

# 删除空几何信息的行
zhejiang_boundary = zhejiang_boundary.dropna(subset=['geometry'])

# 创建一个新的空数据集
new_zhejiang_boundary = gpd.GeoDataFrame()

# 遍历每一行，将非空几何信息的行添加到新数据集中
for index, row in zhejiang_boundary.iterrows():
    new_zhejiang_boundary = pd.concat([new_zhejiang_boundary, gpd.GeoDataFrame([row])], ignore_index=True)

# 保存新数据集到文件
new_zhejiang_boundary.to_file(r'C:\Users\21523\Desktop\cleaned_zhejiang_boundary.geojson', driver='GeoJSON')

# 输出新数据集的摘要信息
print(new_zhejiang_boundary.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      78 non-null     object  
 1   geometry  78 non-null     geometry
dtypes: geometry(1), object(1)
memory usage: 1.3+ KB
None


In [6]:
new_zhejiang_boundary

,name,geometry
0,上城区,"POLYGON ((120.13711 30.19809, 120.13716 30.199..."
1,拱墅区,"POLYGON ((120.08692 30.33302, 120.08782 30.333..."
2,西湖区,"POLYGON ((119.99634 30.18154, 120.00131 30.188..."
3,滨江区,"POLYGON ((120.12439 30.17220, 120.12463 30.177..."
4,萧山区,"POLYGON ((120.07713 29.88752, 120.07716 29.887..."
...,...,...
73,松阳县,"POLYGON ((119.17226 28.30330, 119.17368 28.306..."
74,云和县,"POLYGON ((119.35398 27.97945, 119.35482 27.981..."
75,庆元县,"POLYGON ((118.83593 27.82947, 118.83647 27.830..."
76,景宁畲族自治县,"POLYGON ((119.23581 27.80097, 119.23605 27.805..."


In [7]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape
from scipy.spatial import ConvexHull

# 读取浙江省的边界数据
zhejiang_boundary = gpd.read_file(r'C:/Users/21523/Desktop/cleaned_zhejiang_boundary.geojson')

# 提取奉化区、象山县、北仑区的数据
districts_of_interest = zhejiang_boundary[zhejiang_boundary['name'].isin(['平湖市','海宁市','南浔区','安吉县','吴兴区','临平区', '萧山区', '余杭区','新昌县','奉化区'])]

# 读取数据集，并随机选择10000条数据
data = pd.read_csv(r'D:\第四篇论文数据\第二篇论文编写程序\人类行为时空特性关联性建模\浙江省手机信令数据集\visit_duration_distance_v.csv', usecols=['slon', 'slat']).sample(n=10000)

# 创建地图对象
m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

# 遍历选定的区域
for index, row in districts_of_interest.iterrows():
    # Get the geometry for the district
    geom = row['geometry']
    if geom is None:
        continue
    
    district_boundary = shape(geom)
    
    # 如果数据点为空，跳过绘制
    if data.empty:
        continue
    
    # 创建几何对象的缓冲区来避免拓扑错误，并放大凸包
    district_boundary_buffered = district_boundary.buffer(0.0000000001)
    district_boundary_expanded = district_boundary_buffered.buffer(district_boundary_buffered.bounds[1] * 0.0000000000000001)
    
    # 筛选位于当前区域边界内的数据点
    points_in_district = data[data.apply(lambda x: district_boundary_expanded.contains(Point(x['slon'], x['slat'])), axis=1)]
    
    # 如果数据点为空，跳过绘制
    if points_in_district.empty:
        continue
    
    # 获取数据点的坐标
    coords = points_in_district[['slon', 'slat']].values
    
    # 计算凸包
    hull = ConvexHull(coords)
    
    # 创建凸包多边形的坐标列表
    hull_coords = [(coords[i][1], coords[i][0]) for i in hull.vertices]
    
    # 将凸包多边形添加到地图
    folium.Polygon(locations=hull_coords, fill=True, fill_color='#3186cc', fill_opacity=0.5, color='#3186cc', weight=1).add_to(m)

# 展示地图
m

In [2]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape
from scipy.spatial import ConvexHull

# 读取浙江省的边界数据
zhejiang_boundary = gpd.read_file(r'C:/Users/21523/Desktop/cleaned_zhejiang_boundary.geojson')

# 读取数据集，并随机选择10000条数据
data = pd.read_csv(r'D:\第四篇论文数据\第二篇论文编写程序\人类行为时空特性关联性建模\浙江省手机信令数据集\visit_duration_distance_v.csv', usecols=['slon', 'slat']).sample(n=10000)

# 创建地图对象
m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

# 遍历选定的区域
for index, row in zhejiang_boundary.iterrows():
    # Get the geometry for the district
    geom = row['geometry']
    if geom is None:
        continue
    
    district_boundary = shape(geom)
    
    # 如果数据点为空，跳过绘制
    if data.empty:
        continue
    
    # 创建几何对象的缓冲区来避免拓扑错误，并放大凸包
    district_boundary_buffered = district_boundary.buffer(0)
    district_boundary_expanded = district_boundary_buffered.buffer(district_boundary_buffered.bounds[1] * 0.001)
    
    # 筛选位于当前区域边界内的数据点
    points_in_district = data[data.apply(lambda x: district_boundary_expanded.contains(Point(x['slon'], x['slat'])), axis=1)]
    
    # 如果数据点为空，跳过绘制
    if points_in_district.empty:
        continue
    
    # 获取数据点的坐标
    coords = points_in_district[['slon', 'slat']].values
    
    # 计算凸包
    hull = ConvexHull(coords)
    
    # 创建凸包多边形的坐标列表
    hull_coords = [(coords[i][1], coords[i][0]) for i in hull.vertices]
    
    # 将凸包多边形添加到地图
    folium.Polygon(locations=hull_coords, fill=True, fill_color='#3186cc', fill_opacity=0.5, color='#3186cc', weight=1).add_to(m)

# 展示地图
m

In [8]:
zhejiang_boundary = gpd.read_file(r'C:/Users/21523/Desktop/cleaned_zhejiang_boundary.geojson')

In [9]:
zhejiang_boundary

,name,geometry
0,上城区,"POLYGON ((120.13711 30.19809, 120.13716 30.199..."
1,拱墅区,"POLYGON ((120.08692 30.33302, 120.08782 30.333..."
2,西湖区,"POLYGON ((119.99634 30.18154, 120.00131 30.188..."
3,滨江区,"POLYGON ((120.12439 30.17220, 120.12463 30.177..."
4,萧山区,"POLYGON ((120.07713 29.88752, 120.07716 29.887..."
...,...,...
73,松阳县,"POLYGON ((119.17226 28.30330, 119.17368 28.306..."
74,云和县,"POLYGON ((119.35398 27.97945, 119.35482 27.981..."
75,庆元县,"POLYGON ((118.83593 27.82947, 118.83647 27.830..."
76,景宁畲族自治县,"POLYGON ((119.23581 27.80097, 119.23605 27.805..."


In [10]:
zhejiang_boundary = zhejiang_boundary[zhejiang_boundary['name'] != '临安区']

In [11]:
zhejiang_boundary

,name,geometry
0,上城区,"POLYGON ((120.13711 30.19809, 120.13716 30.199..."
1,拱墅区,"POLYGON ((120.08692 30.33302, 120.08782 30.333..."
2,西湖区,"POLYGON ((119.99634 30.18154, 120.00131 30.188..."
3,滨江区,"POLYGON ((120.12439 30.17220, 120.12463 30.177..."
4,萧山区,"POLYGON ((120.07713 29.88752, 120.07716 29.887..."
...,...,...
73,松阳县,"POLYGON ((119.17226 28.30330, 119.17368 28.306..."
74,云和县,"POLYGON ((119.35398 27.97945, 119.35482 27.981..."
75,庆元县,"POLYGON ((118.83593 27.82947, 118.83647 27.830..."
76,景宁畲族自治县,"POLYGON ((119.23581 27.80097, 119.23605 27.805..."


In [15]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape
from scipy.spatial import ConvexHull
import os
import time
import random

# 读取浙江省的边界数据
zhejiang_boundary = gpd.read_file(r'C:/Users/21523/Desktop/cleaned_zhejiang_boundary.geojson')

# 定义保存路径
save_path = r'C:\Users\21523\Desktop\区级图'

# 创建地图对象
m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

# 确保保存路径存在
os.makedirs(save_path, exist_ok=True)

# 循环生成大图并保存
while True:
    # 随机选择 30 到 40 个区
    selected_districts = random.sample(range(len(zhejiang_boundary)), random.randint(40, 60))
    
    # 创建空列表存储所有凸包坐标
    all_hulls = []
    
    # 循环遍历选定的区域
    for idx in selected_districts:
        row = zhejiang_boundary.iloc[idx]
        
        # 获取区域的几何信息
        geom = row['geometry']
        if geom is None:
            continue
        
        district_boundary = shape(geom)
        
        # 筛选位于当前区域边界内的数据点
        points_in_district = data[data.apply(lambda x: district_boundary.contains(Point(x['slon'], x['slat'])), axis=1)]
        
        # 如果数据点为空，跳过绘制
        if points_in_district.empty:
            continue
        
        # 获取数据点的坐标
        coords = points_in_district[['slon', 'slat']].values
        
        # 计算凸包
        hull = ConvexHull(coords)
        
        # 调整凸包大小的比例因子
        scale_factor = 0.15
        
        # 创建凸包多边形的坐标列表，并对其进行缩小处理
        hull_coords_scaled = []
        for i in hull.vertices:
            # 获取原始坐标
            original_coord = coords[i]
            
            # 计算新的缩小后的坐标
            scaled_lat = original_coord[1] + scale_factor * (coords[hull.vertices[0]][1] - original_coord[1])
            scaled_lon = original_coord[0] + scale_factor * (coords[hull.vertices[0]][0] - original_coord[0])
            
            # 添加缩小后的坐标到列表中
            hull_coords_scaled.append((scaled_lat, scaled_lon))
        
        # 添加凸包坐标到列表中
        all_hulls.append(hull_coords_scaled)

    # 将所有凸包绘制在地图上
    for hull_coords in all_hulls:
        folium.Polygon(locations=hull_coords, fill=True, fill_color='#ff0000', fill_opacity=0.5, color='#ff0000', weight=1).add_to(m)
    
    # 获取当前时间戳
    timestamp = int(time.time())
    
    # 构建保存文件名
    html_filename = os.path.join(save_path, f'zhejiang_all_districts_{timestamp}.html')
    
    # 保存地图为HTML格式
    m.save(html_filename)
    
    # 清空地图以准备下一次循环
    m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

KeyboardInterrupt: 

In [ ]:
zhejiang_boundary = gpd.read_file(r'C:\Users\21523\Desktop\全国乡镇边界的经纬度数据集\浙江省行政区域经纬度数据集\scaled_40_percent_浙江省.geoJson')

In [7]:
import folium
import random

# 浙江省地级市边界经纬度
zhejiang_cities_boundaries = {
    "Hangzhou": [(30.1169, 119.7835), (30.5234, 120.4632)],
    "Ningbo": [(28.7426, 120.5204), (30.0942, 121.9323)],
    "Wenzhou": [(27.1683, 119.7286), (28.6604, 121.2928)],
    "Jiaxing": [(30.4383, 120.4856), (31.0759, 121.1799)],
    "Huzhou": [(29.6414, 119.7597), (30.3547, 120.3778)],
    "Shaoxing": [(29.0574, 120.0054), (30.2678, 121.0008)],
    "Jinhua": [(28.3202, 118.6362), (29.2475, 120.0364)],
    "Quzhou": [(27.3619, 118.3114), (29.1075, 119.9602)],
    "Zhoushan": [(29.4907, 121.0466), (30.2804, 122.2742)],
    "Taizhou": [(28.0311, 120.3526), (29.1231, 122.2111)],
    "Lishui": [(27.2468, 118.4674), (28.9013, 120.0846)]
}

# 创建地图
m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

# 生成每个地级市的五个范围圈
for city, boundary in zhejiang_cities_boundaries.items():
    # 获取地级市边界范围
    min_lat, min_lon = boundary[0]
    max_lat, max_lon = boundary[1]
    for _ in range(5):
        # 随机生成圆的半径和颜色
        radius = random.uniform(1000, 15000)  # 单位：米
        color = '#' + ''.join(random.choices('0123456789ABCDEF', k=6))

        # 在地级市范围内随机生成点
        lat = random.uniform(min_lat, max_lat)
        lon = random.uniform(min_lon, max_lon)

        # 创建范围圈并添加到地图上
        circle = folium.Circle(location=[lat, lon], radius=radius, color='#ff0000', fill=True).add_to(m)

# 显示地图
m

In [21]:
import folium
import random
from scipy.spatial import ConvexHull

# 随机生成35组经纬度点，每组点数为3个以上，经纬度范围不太大
coordinates = []
for _ in range(35):
    num_points = random.randint(3, 10)  # 生成3到10个点
    lat_center = random.uniform(28.0, 30.0)  # 设置纬度中心
    lon_center = random.uniform(119.0, 121.0)  # 设置经度中心
    coords = [(random.uniform(lat_center - 0.2, lat_center + 0.2), 
               random.uniform(lon_center - 0.2, lon_center + 0.2)) for _ in range(num_points)]
    coordinates.append(coords)

# 创建地图
m = folium.Map(location=[29.0842, 119.6474], zoom_start=8, tiles='CartoDB positron')

# 绘制35个凸包
for i, coords in enumerate(coordinates, start=1):
    # 如果坐标点少于3个，则不绘制凸包
    if len(coords) < 3:
        continue
    # 计算凸包
    hull = ConvexHull(coords)
    # 凸包的顶点
    hull_coords = [coords[vertex] for vertex in hull.vertices]
    # 添加第i个凸包到地图
    folium.Polygon(locations=hull_coords, color='#ff0000', fill=True, weight=1).add_to(m)

# 显示地图
m